In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from collections import Counter
import holidays
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#### read files
df_train = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/train.csv")
df_log = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/view_log.csv")
df_item = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/train_NA17Sgz/item_data.csv")

In [3]:
df_test = pd.read_csv("/Users/s0c02nj/Desktop/WNS2/test.csv")

In [4]:
df_train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [5]:
df_log.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [6]:
df_item.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [7]:
df_log_item = pd.merge(left = df_log, 
                  right= df_item, 
                  how='left',
                  left_on=['item_id'], 
                  right_on=['item_id'])

In [8]:
df_log_item.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,android,503590,74788,7640,1376.0,7.0,71.0,228.0,545.0
2,2018-10-15 08:58:00,android,573960,23628,128855,4544.0,4.0,38.0,62.0,5609.0
3,2018-10-15 08:58:00,android,121691,2430,12774,904.0,17.0,39.0,252.0,2740.0
4,2018-10-15 08:58:00,android,218564,19227,28296,2304.0,12.0,57.0,54.0,7422.0


In [9]:
df_log_item.nunique()

server_time       82840
device_type           3
session_id      1014970
user_id           89157
item_id          126708
item_price        13207
category_1           17
category_2           79
category_3          335
product_type       7846
dtype: int64

In [10]:
cat_cols = ['device_type', 'category_1', 'category_2', 'category_3', 'product_type', 'item_id']

for col in cat_cols:
    le = preprocessing.LabelEncoder()
    df_log_item[col] = le.fit_transform(df_log_item[col].astype(str))

In [11]:
df_log_item.nunique()

server_time       82840
device_type           3
session_id      1014970
user_id           89157
item_id          126708
item_price        13207
category_1           18
category_2           80
category_3          336
product_type       7847
dtype: int64

In [12]:
df_log_item["cat1_cat2"] = df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str")
df_log_item["cat1_cat3"] = df_log_item["category_1"].astype("str") + df_log_item["category_3"].astype("str")
df_log_item["cat2_cat3"] = df_log_item["category_2"].astype("str") + df_log_item["category_3"].astype("str")
df_log_item["cat1_cat2_cat3"] = df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str") + df_log_item["category_3"].astype("str")

In [13]:
us_holidays = holidays.UnitedStates()

In [14]:
##### server feature week, month, day and hour
df_log_item['server_time'] = pd.to_datetime(df_log_item['server_time'])

df_log_item['server_wk'] = df_log_item['server_time'].dt.week
df_log_item['server_mth'] = df_log_item['server_time'].dt.month
df_log_item['server_day'] = df_log_item['server_time'].dt.day
df_log_item['server_hr'] = df_log_item['server_time'].dt.hour
df_log_item['server_weekend'] = (df_log_item['server_time'].dt.weekday >= 5).astype(int)

In [15]:
### impute price
df_log_item['item_price'] = df_log_item['item_price'].apply(lambda x:np.log1p(x))
price_med = df_log_item['item_price'].median()
df_log_item['item_price'] = df_log_item['item_price'].replace(np.nan, price_med, regex = True)
df_log_item['item_price_inv'] = df_log_item['item_price'].apply(lambda x: 1/(1+x))
df_log_item['item_price_sq'] = df_log_item['item_price'].apply(lambda x: x**2)


In [16]:
def check_nas(df):
    print(df.isna().sum())

In [17]:
def q3(x):
    return np.percentile(x , 75)

def q1(x):
    return np.percentile(x , 25)

def mode(x):
    return Counter(x).most_common(1)[0][0]

def mode_1(x):
    try:
        return Counter(x).most_common(2)[1][0]
    except:
        return 999999

def mode_2(x):    
    try:
        return Counter(x).most_common(3)[2][0]
    except:
        return 999999

In [18]:
x = [1,1,1,1,1,1,1,2,2,2,2,3]

In [19]:
mode_1(x)

2

In [20]:
is_click = df_train["is_click"]
df_train = df_train.drop(["is_click"], axis=1)

In [21]:
df_combined = pd.concat([df_train, df_test])

In [22]:
aggs = {}
aggs['item_price'] = ['median', 'mean', 'sum', 'max', 'min', q1, q3,  mode, mode_1, mode_2]
aggs['device_type'] = ['count', 'nunique', mode, mode_1, mode_2]
aggs['user_id'] = ['count', 'nunique']
aggs['session_id'] = ['count', 'nunique']
aggs['item_id'] = ['nunique', mode, mode_1, mode_2]
aggs['category_1'] = ['nunique', mode, mode_1, mode_2]
aggs['category_2'] = ['nunique', mode, mode_1, mode_2]
aggs['category_3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat2'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat2_cat3'] = ['nunique', mode, mode_1, mode_2]
aggs['cat1_cat2_cat3'] = ['nunique', mode, mode_1, mode_2]

aggs['server_wk'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_mth'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_day'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_hr'] = ['nunique', mode, mode_1, mode_2, 'mean']
aggs['server_weekend'] = ['nunique', mode, mode_1, mode_2, 'sum']

In [23]:
for col in tqdm(["server_day", "server_hr",'category_1','category_2']):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    df_log_item = pd.merge(left = df_log_item, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 4/4 [08:10<00:00, 125.84s/it]


In [24]:
for col in tqdm(["user_id"]):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = df_log_item.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    df_final = pd.merge(left = df_combined, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 1/1 [03:33<00:00, 213.31s/it]


In [25]:
df_train.shape[0] == df_final.shape[0]

False

In [26]:
df_final.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,user_id_item_price_median,user_id_item_price_mean,user_id_item_price_sum,user_id_item_price_max,...,user_id_server_hr_nunique,user_id_server_hr_mode,user_id_server_hr_mode_1,user_id_server_hr_mode_2,user_id_server_hr_mean,user_id_server_weekend_nunique,user_id_server_weekend_mode,user_id_server_weekend_mode_1,user_id_server_weekend_mode_2,user_id_server_weekend_sum
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,7.762596,7.762596,7.762596,7.762596,...,1,18,999999,999999,18.000000,1,0,999999,999999,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,8.137980,8.199240,98.390874,9.558671,...,7,5,19,6,11.750000,2,0,1,999999,4
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,7.294462,7.294462,14.588923,7.707512,...,1,12,999999,999999,12.000000,1,0,999999,999999,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,7.695744,7.971887,143.493966,11.446454,...,7,18,12,19,16.444444,2,0,1,999999,5
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,8.187685,8.094990,372.369547,11.673146,...,13,22,23,5,16.065217,2,0,1,999999,5


In [27]:
df_final['impression_time'] = pd.to_datetime(df_final['impression_time'])

df_final['impression_wk'] = df_final['impression_time'].dt.week
df_final['impression_mth'] = df_final['impression_time'].dt.month
df_final['impression_day'] = df_final['impression_time'].dt.day
df_final['impression_hr'] = df_final['impression_time'].dt.hour

df_final['impression_weekend'] = (df_final['impression_time'].dt.weekday >= 5).astype(int)

Reading Item Vector

In [28]:
df_embed = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/golden_feats/user_embed_20.csv')

In [29]:
col = 'user_id'
df_final1 = pd.merge(left = df_final, right= df_embed, how='left',
                    left_on=[col], right_on=[col])

In [30]:
df_final = df_final.drop(['impression_id', 'impression_time', 'user_id'], axis =1)

In [31]:
df_final1 = df_final1.drop(['impression_id', 'impression_time', 'user_id'], axis =1)

In [32]:
col_counts = []

for col in tqdm(df_final.columns):
    
    counter = Counter(df_final[col])
    col_counts.append(str(col)+'_count')
    
    df_final1[str(col)+'_count'] = df_final1[col].apply(lambda x:counter[x])

100%|██████████| 82/82 [00:13<00:00,  7.21it/s]


In [33]:
df_final = df_final1

In [34]:
cat_appended_cols = []
for col in df_final.columns:
    if df_final[col].dtype == "object":
        cat_appended_cols.append(col)
        print(col , df_final[col].dtype)

os_version object
user_id_cat1_cat2_mode object
user_id_cat1_cat2_mode_1 object
user_id_cat1_cat2_mode_2 object
user_id_cat1_cat3_mode object
user_id_cat1_cat3_mode_1 object
user_id_cat1_cat3_mode_2 object
user_id_cat2_cat3_mode object
user_id_cat2_cat3_mode_1 object
user_id_cat2_cat3_mode_2 object
user_id_cat1_cat2_cat3_mode object
user_id_cat1_cat2_cat3_mode_1 object
user_id_cat1_cat2_cat3_mode_2 object


In [36]:
cat_cols_train = ['app_code', 'os_version'] + cat_appended_cols

for col in cat_cols_train:
    print(col)
    le = preprocessing.LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col].astype(int))

app_code
os_version
os_version
user_id_cat1_cat2_mode
user_id_cat1_cat2_mode_1
user_id_cat1_cat2_mode_2
user_id_cat1_cat3_mode
user_id_cat1_cat3_mode_1
user_id_cat1_cat3_mode_2
user_id_cat2_cat3_mode
user_id_cat2_cat3_mode_1
user_id_cat2_cat3_mode_2
user_id_cat1_cat2_cat3_mode
user_id_cat1_cat2_cat3_mode_1
user_id_cat1_cat2_cat3_mode_2


In [37]:
df_final.head()

,app_code,os_version,is_4G,user_id_item_price_median,user_id_item_price_mean,user_id_item_price_sum,user_id_item_price_max,user_id_item_price_min,user_id_item_price_q1,user_id_item_price_q3,...,user_id_server_weekend_nunique_count,user_id_server_weekend_mode_count,user_id_server_weekend_mode_1_count,user_id_server_weekend_mode_2_count,user_id_server_weekend_sum_count,impression_wk_count,impression_mth_count,impression_day_count,impression_hr_count,impression_weekend_count
0,421,2,0,7.762596,7.762596,7.762596,7.762596,7.762596,7.762596,7.762596,...,69589,269441,69589,328284,57584,28702,131317,24078,28834,237139
1,466,1,1,8.137980,8.199240,98.390874,9.558671,6.932448,8.072265,8.401682,...,258695,269441,211857,328284,15867,28702,131317,24078,28834,237139
2,259,0,1,7.294462,7.294462,14.588923,7.707512,6.881411,7.087937,7.500987,...,69589,269441,69589,328284,57584,28702,131317,24078,28834,237139
3,244,1,1,7.695744,7.971887,143.493966,11.446454,5.521461,7.379008,8.898502,...,258695,269441,211857,328284,14373,28702,131317,24078,28834,237139
4,472,1,0,8.187685,8.094990,372.369547,11.673146,5.666427,7.084755,8.530450,...,258695,269441,211857,328284,14373,28702,131317,24078,28834,237139


In [38]:
x_train = df_final[0 : df_train.shape[0]]
x_test = df_final[df_train.shape[0] : ]
y = is_click

In [39]:
x_t, x_v, y_t, y_v = train_test_split(x_train, y, stratify=y, random_state=42, test_size=0.2)

In [45]:
params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth': 20,  
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary', 
    'verbosity': 1,
    'num_leaves': 30,
    'bagging_fraction':0.7, #rows
    'bagging_frequency':10,
    'min_data_in_leaf':30,
    'feature_fraction':0.2, # features
    'lambda_l1':10,
    'lambda_l2':10,
    'min_gain_split':0.3
}

In [46]:
d_train = lgb.Dataset(x_t, label=y_t)
d_val = lgb.Dataset(x_v, label=y_v)

clf = lgb.train(params, d_train, 150000, verbose_eval=1, 
                valid_sets = [d_train, d_val], early_stopping_rounds = 500)

[1]	training's auc: 0.6949	valid_1's auc: 0.671937
Training until validation scores don't improve for 500 rounds.
[2]	training's auc: 0.700156	valid_1's auc: 0.673985
[3]	training's auc: 0.714368	valid_1's auc: 0.693115
[4]	training's auc: 0.715248	valid_1's auc: 0.694209
[5]	training's auc: 0.715194	valid_1's auc: 0.693509
[6]	training's auc: 0.715111	valid_1's auc: 0.690875
[7]	training's auc: 0.715156	valid_1's auc: 0.688699
[8]	training's auc: 0.713017	valid_1's auc: 0.685298
[9]	training's auc: 0.71806	valid_1's auc: 0.692429
[10]	training's auc: 0.716565	valid_1's auc: 0.689883
[11]	training's auc: 0.715076	valid_1's auc: 0.687592
[12]	training's auc: 0.713264	valid_1's auc: 0.685165
[13]	training's auc: 0.711621	valid_1's auc: 0.682782
[14]	training's auc: 0.714552	valid_1's auc: 0.68716
[15]	training's auc: 0.71356	valid_1's auc: 0.685375
[16]	training's auc: 0.716694	valid_1's auc: 0.689516
[17]	training's auc: 0.715272	valid_1's auc: 0.687865
[18]	training's auc: 0.714268	val

[160]	training's auc: 0.745822	valid_1's auc: 0.71164
[161]	training's auc: 0.745918	valid_1's auc: 0.711637
[162]	training's auc: 0.746292	valid_1's auc: 0.712015
[163]	training's auc: 0.746288	valid_1's auc: 0.711942
[164]	training's auc: 0.746408	valid_1's auc: 0.711943
[165]	training's auc: 0.746506	valid_1's auc: 0.711867
[166]	training's auc: 0.746518	valid_1's auc: 0.711809
[167]	training's auc: 0.746771	valid_1's auc: 0.712135
[168]	training's auc: 0.747067	valid_1's auc: 0.712435
[169]	training's auc: 0.747179	valid_1's auc: 0.712469
[170]	training's auc: 0.747272	valid_1's auc: 0.712503
[171]	training's auc: 0.747632	valid_1's auc: 0.712917
[172]	training's auc: 0.74769	valid_1's auc: 0.712949
[173]	training's auc: 0.748008	valid_1's auc: 0.713288
[174]	training's auc: 0.748293	valid_1's auc: 0.713578
[175]	training's auc: 0.748331	valid_1's auc: 0.713586
[176]	training's auc: 0.74835	valid_1's auc: 0.713518
[177]	training's auc: 0.748392	valid_1's auc: 0.713506
[178]	trainin

[318]	training's auc: 0.765439	valid_1's auc: 0.725576
[319]	training's auc: 0.765554	valid_1's auc: 0.725711
[320]	training's auc: 0.765586	valid_1's auc: 0.725735
[321]	training's auc: 0.76565	valid_1's auc: 0.725799
[322]	training's auc: 0.765748	valid_1's auc: 0.725898
[323]	training's auc: 0.765793	valid_1's auc: 0.725875
[324]	training's auc: 0.765873	valid_1's auc: 0.725892
[325]	training's auc: 0.766016	valid_1's auc: 0.725946
[326]	training's auc: 0.766107	valid_1's auc: 0.726043
[327]	training's auc: 0.766226	valid_1's auc: 0.726079
[328]	training's auc: 0.766319	valid_1's auc: 0.726104
[329]	training's auc: 0.766462	valid_1's auc: 0.726216
[330]	training's auc: 0.766517	valid_1's auc: 0.726196
[331]	training's auc: 0.766635	valid_1's auc: 0.726313
[332]	training's auc: 0.766685	valid_1's auc: 0.72631
[333]	training's auc: 0.766763	valid_1's auc: 0.726425
[334]	training's auc: 0.766854	valid_1's auc: 0.726468
[335]	training's auc: 0.766926	valid_1's auc: 0.726515
[336]	traini

[474]	training's auc: 0.778469	valid_1's auc: 0.732601
[475]	training's auc: 0.778535	valid_1's auc: 0.732703
[476]	training's auc: 0.778587	valid_1's auc: 0.732732
[477]	training's auc: 0.778632	valid_1's auc: 0.732778
[478]	training's auc: 0.778721	valid_1's auc: 0.732797
[479]	training's auc: 0.778783	valid_1's auc: 0.732824
[480]	training's auc: 0.778845	valid_1's auc: 0.732848
[481]	training's auc: 0.778934	valid_1's auc: 0.732876
[482]	training's auc: 0.779024	valid_1's auc: 0.732896
[483]	training's auc: 0.779101	valid_1's auc: 0.732936
[484]	training's auc: 0.779134	valid_1's auc: 0.732991
[485]	training's auc: 0.779206	valid_1's auc: 0.733056
[486]	training's auc: 0.779226	valid_1's auc: 0.733062
[487]	training's auc: 0.779302	valid_1's auc: 0.733064
[488]	training's auc: 0.779349	valid_1's auc: 0.733073
[489]	training's auc: 0.779421	valid_1's auc: 0.733085
[490]	training's auc: 0.779519	valid_1's auc: 0.733085
[491]	training's auc: 0.779572	valid_1's auc: 0.733121
[492]	trai

[625]	training's auc: 0.788799	valid_1's auc: 0.736508
[626]	training's auc: 0.788867	valid_1's auc: 0.736524
[627]	training's auc: 0.788941	valid_1's auc: 0.736548
[628]	training's auc: 0.789014	valid_1's auc: 0.73659
[629]	training's auc: 0.789105	valid_1's auc: 0.736619
[630]	training's auc: 0.789158	valid_1's auc: 0.736629
[631]	training's auc: 0.789247	valid_1's auc: 0.736633
[632]	training's auc: 0.78932	valid_1's auc: 0.736647
[633]	training's auc: 0.789376	valid_1's auc: 0.736675
[634]	training's auc: 0.789426	valid_1's auc: 0.736697
[635]	training's auc: 0.789478	valid_1's auc: 0.736711
[636]	training's auc: 0.789558	valid_1's auc: 0.736739
[637]	training's auc: 0.789615	valid_1's auc: 0.736745
[638]	training's auc: 0.789683	valid_1's auc: 0.736747
[639]	training's auc: 0.789743	valid_1's auc: 0.736771
[640]	training's auc: 0.789852	valid_1's auc: 0.736821
[641]	training's auc: 0.789912	valid_1's auc: 0.736833
[642]	training's auc: 0.789962	valid_1's auc: 0.736868
[643]	traini

...

[4888]	training's auc: 0.904429	valid_1's auc: 0.755664
[4889]	training's auc: 0.904441	valid_1's auc: 0.755666
[4890]	training's auc: 0.904452	valid_1's auc: 0.755668
[4891]	training's auc: 0.904464	valid_1's auc: 0.755669
[4892]	training's auc: 0.904476	valid_1's auc: 0.75568
[4893]	training's auc: 0.904494	valid_1's auc: 0.755671
[4894]	training's auc: 0.904512	valid_1's auc: 0.755669
[4895]	training's auc: 0.904524	valid_1's auc: 0.755669
[4896]	training's auc: 0.904537	valid_1's auc: 0.755668
[4897]	training's auc: 0.904548	valid_1's auc: 0.755669
[4898]	training's auc: 0.90456	valid_1's auc: 0.755663
[4899]	training's auc: 0.904573	valid_1's auc: 0.755664
[4900]	training's auc: 0.90459	valid_1's auc: 0.755666
[4901]	training's auc: 0.904595	valid_1's auc: 0.755671
[4902]	training's auc: 0.904611	valid_1's auc: 0.755681
[4903]	training's auc: 0.904627	valid_1's auc: 0.755677
[4904]	training's auc: 0.904641	valid_1's auc: 0.755684
[4905]	training's auc: 0.904659	valid_1's auc: 0.75

[5041]	training's auc: 0.906503	valid_1's auc: 0.755779
[5042]	training's auc: 0.906515	valid_1's auc: 0.755775
[5043]	training's auc: 0.906526	valid_1's auc: 0.755772
[5044]	training's auc: 0.906541	valid_1's auc: 0.755774
[5045]	training's auc: 0.906558	valid_1's auc: 0.755775
[5046]	training's auc: 0.90657	valid_1's auc: 0.755765
[5047]	training's auc: 0.906582	valid_1's auc: 0.755773
[5048]	training's auc: 0.906598	valid_1's auc: 0.755768
[5049]	training's auc: 0.906601	valid_1's auc: 0.755764
[5050]	training's auc: 0.906614	valid_1's auc: 0.755762
[5051]	training's auc: 0.906627	valid_1's auc: 0.755765
[5052]	training's auc: 0.90664	valid_1's auc: 0.755765
[5053]	training's auc: 0.906646	valid_1's auc: 0.755763
[5054]	training's auc: 0.90666	valid_1's auc: 0.755758
[5055]	training's auc: 0.906668	valid_1's auc: 0.755763
[5056]	training's auc: 0.90668	valid_1's auc: 0.755759
[5057]	training's auc: 0.906697	valid_1's auc: 0.755753
[5058]	training's auc: 0.906699	valid_1's auc: 0.755

[5191]	training's auc: 0.908379	valid_1's auc: 0.755656
[5192]	training's auc: 0.908394	valid_1's auc: 0.755648
[5193]	training's auc: 0.908404	valid_1's auc: 0.755652
[5194]	training's auc: 0.908422	valid_1's auc: 0.755656
[5195]	training's auc: 0.908432	valid_1's auc: 0.755657
[5196]	training's auc: 0.908441	valid_1's auc: 0.755662
[5197]	training's auc: 0.908459	valid_1's auc: 0.755664
[5198]	training's auc: 0.908474	valid_1's auc: 0.755655
[5199]	training's auc: 0.908483	valid_1's auc: 0.755663
[5200]	training's auc: 0.908502	valid_1's auc: 0.755661
[5201]	training's auc: 0.908519	valid_1's auc: 0.755664
[5202]	training's auc: 0.90853	valid_1's auc: 0.755661
[5203]	training's auc: 0.908543	valid_1's auc: 0.755664
[5204]	training's auc: 0.908558	valid_1's auc: 0.755669
[5205]	training's auc: 0.908574	valid_1's auc: 0.755668
[5206]	training's auc: 0.908589	valid_1's auc: 0.755655
[5207]	training's auc: 0.90861	valid_1's auc: 0.755657
[5208]	training's auc: 0.908624	valid_1's auc: 0.7

[5339]	training's auc: 0.910263	valid_1's auc: 0.755562
[5340]	training's auc: 0.910272	valid_1's auc: 0.755568
[5341]	training's auc: 0.910279	valid_1's auc: 0.755565
[5342]	training's auc: 0.910289	valid_1's auc: 0.755562
[5343]	training's auc: 0.9103	valid_1's auc: 0.755564
[5344]	training's auc: 0.910319	valid_1's auc: 0.755576
[5345]	training's auc: 0.910338	valid_1's auc: 0.755573
[5346]	training's auc: 0.910349	valid_1's auc: 0.755572
[5347]	training's auc: 0.910361	valid_1's auc: 0.755576
[5348]	training's auc: 0.910377	valid_1's auc: 0.755587
[5349]	training's auc: 0.910384	valid_1's auc: 0.755586
[5350]	training's auc: 0.910393	valid_1's auc: 0.755583
[5351]	training's auc: 0.910407	valid_1's auc: 0.755588
[5352]	training's auc: 0.910424	valid_1's auc: 0.755589
[5353]	training's auc: 0.910431	valid_1's auc: 0.755588
[5354]	training's auc: 0.910447	valid_1's auc: 0.755585
[5355]	training's auc: 0.910457	valid_1's auc: 0.755586
[5356]	training's auc: 0.910477	valid_1's auc: 0.7

[5487]	training's auc: 0.91204	valid_1's auc: 0.755685
[5488]	training's auc: 0.912045	valid_1's auc: 0.755685
[5489]	training's auc: 0.912056	valid_1's auc: 0.755685
[5490]	training's auc: 0.912068	valid_1's auc: 0.755688
[5491]	training's auc: 0.912078	valid_1's auc: 0.755691
[5492]	training's auc: 0.912088	valid_1's auc: 0.755701
[5493]	training's auc: 0.912106	valid_1's auc: 0.755716
[5494]	training's auc: 0.912122	valid_1's auc: 0.755715
[5495]	training's auc: 0.912134	valid_1's auc: 0.755709
[5496]	training's auc: 0.912148	valid_1's auc: 0.755704
[5497]	training's auc: 0.912163	valid_1's auc: 0.755704
[5498]	training's auc: 0.912174	valid_1's auc: 0.755704
[5499]	training's auc: 0.912183	valid_1's auc: 0.755704
[5500]	training's auc: 0.912199	valid_1's auc: 0.755709
[5501]	training's auc: 0.912205	valid_1's auc: 0.755713
[5502]	training's auc: 0.912216	valid_1's auc: 0.75572
[5503]	training's auc: 0.912223	valid_1's auc: 0.755728
[5504]	training's auc: 0.912238	valid_1's auc: 0.7

In [42]:
predictions = clf.predict(x_test)

In [ ]:
sub_df = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/sample_submission.csv')
sub_df['is_click'] = predictions
sub_df.to_csv('/Users/s0c02nj/Desktop/WNS2/output_submission/user_feats.csv', index = False)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), 
                                      x_train.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
feature_imp

In [ ]:
new_feat_imp = feature_imp[feature_imp['Value'] >84]

In [ ]:
list_new_feat = list(new_feat_imp['Feature'])

In [ ]:
#Stratified K-fold

In [48]:
kf = StratifiedKFold(n_splits=12, random_state=42, shuffle=True)

In [49]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(x_train, y)):
    
    d_train = lgb.Dataset(x_train.iloc[train_index], label=y[train_index])
    d_val = lgb.Dataset(x_train.iloc[valid_index], label=y[valid_index])

    clf = lgb.train(params, d_train, 150000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 500)
    

    
    num_models += 1
    
    predictions += clf.predict(x_test)

0it [00:00, ?it/s]

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.805663	valid_1's auc: 0.752132
[2000]	training's auc: 0.840957	valid_1's auc: 0.758469
[3000]	training's auc: 0.864394	valid_1's auc: 0.761279
[4000]	training's auc: 0.882115	valid_1's auc: 0.76228
[5000]	training's auc: 0.896112	valid_1's auc: 0.76262
Early stopping, best iteration is:
[5107]	training's auc: 0.897474	valid_1's auc: 0.762836


1it [02:14, 134.49s/it]

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.805127	valid_1's auc: 0.746779
[2000]	training's auc: 0.840662	valid_1's auc: 0.754491
[3000]	training's auc: 0.864243	valid_1's auc: 0.757243
[4000]	training's auc: 0.882012	valid_1's auc: 0.758378
Early stopping, best iteration is:
[4201]	training's auc: 0.885151	valid_1's auc: 0.758613


2it [04:09, 128.54s/it]

...

10it [23:05, 143.02s/it]

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.805471	valid_1's auc: 0.74921
[2000]	training's auc: 0.841198	valid_1's auc: 0.755276
[3000]	training's auc: 0.864604	valid_1's auc: 0.757393
[4000]	training's auc: 0.882159	valid_1's auc: 0.758153
Early stopping, best iteration is:
[4178]	training's auc: 0.884842	valid_1's auc: 0.758309


11it [24:55, 133.12s/it]

Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.805842	valid_1's auc: 0.747011
[2000]	training's auc: 0.840742	valid_1's auc: 0.754933
[3000]	training's auc: 0.863717	valid_1's auc: 0.758287
[4000]	training's auc: 0.881644	valid_1's auc: 0.760429
[5000]	training's auc: 0.896031	valid_1's auc: 0.761091
[6000]	training's auc: 0.90815	valid_1's auc: 0.76177
[7000]	training's auc: 0.918048	valid_1's auc: 0.762002
Early stopping, best iteration is:
[7336]	training's auc: 0.921067	valid_1's auc: 0.762251


12it [28:01, 148.81s/it]


In [50]:
predictions_12_set = predictions / 12.0

In [51]:
sub_df = pd.read_csv('/Users/s0c02nj/Desktop/WNS2/sample_submission.csv')
sub_df['is_click'] = predictions_12_set
sub_df.to_csv('/Users/s0c02nj/Desktop/WNS2/output_submission/user_embed_cat_agg.csv', index = False)

In [ ]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df['is_click'] = pred2
sub_df.to_csv('sub_agg_time_and_timeagg_count_top3_8654_75389_lr01_10_5_div2fold.csv', index = False)

In [ ]:
temp = df_log_item[df_log_item.user_id == 4557].groupby(['user_id', 'session_id'])['item_id'].apply(list)

In [ ]:
a = []
for user_id in tqdm(list(df_log_item["user_id"].unique())):
    temp = df_log_item[df_log_item.user_id == user_id].groupby(['user_id', 'session_id'])['item_id'].apply(list)
    a += list(temp)

In [ ]:
session_length = [len(t) for t in tqdm(a)]

In [ ]:
np.percentile(session_length, 75)

In [ ]:
Counter(session_length)

In [ ]:
df_log_item.columns

In [ ]:
df_feats = pd.merge(left = df_train, 
                  right= df_log_item, 
                  how='left',
                  left_on=['user_id'], 
                  right_on=['user_id'])

In [ ]:
df_log_item.isna().sum()

In [ ]:
df_test_feats = pd.merge(left = df_test, 
                  right= df_log_item, 
                  how='left',
                  left_on=['user_id'], 
                  right_on=['user_id'])

In [ ]:
df_feats.columns

In [ ]:
categorical_features = ["app_code", "os_version"] + ['device_type', 'session_id', 'category_1', 'category_2', 'category_3', 'product_type', 'cat1_cat2', 'cat1_cat3', 'cat2_cat3', 'cat1_cat2_cat3']

In [ ]:
Counter(predictions)

In [ ]:
df_test["is_click"] = predictions

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), 
                                      x_train.columns)), columns=['Value','Feature'])

In [ ]:
feature_imp

In [ ]:
df_test.head()

In [ ]:
sub = df_test[["impression_id", "is_click"]]
sub.to_csv("no_extra_feats_1.csv", index=False)

In [ ]:
df_log_item["category_1"].astype("str") + df_log_item["category_2"].astype("str")

In [ ]:
def hyper_params(md,n_l,bf,bf1,mdl,ff,l1,l2,mgs):
    
    params = {
        'boost': 'gbdt',
        'learning_rate': 0.03,
        'max_depth': md,
        'metric':{'auc'},
        'num_threads': -1,
        'objective': 'binary', 
        'verbosity': 1,
        'num_leaves': n_l,
        'bagging_fraction':bf, #rows
        'bagging_frequency':bf1,
        'min_data_in_leaf':mdl,
        'feature_fraction':ff, # features
        'lambda_l1':l1,
        'lambda_l2':l2,
        'min_gain_split':mgs,
        'is_unbalance':True}
    
    return params

In [ ]:
md_range = np.arange(6,30,1)


In [ ]:
params = {
    'boost': 'gbdt',
    'learning_rate': 0.03,
    'max_depth': 8,  
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary', 
    'verbosity': 1,
    'num_leaves': 8,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':10,
    'min_data_in_leaf':3,
    'feature_fraction':0.35, # features
    'lambda_l1':5,
    'lambda_l2':10,
    'min_gain_split':0.3,
    'is_unbalance':True
}